In [1]:
from scipy import sparse
import pandas as pd
import numpy as np

## 1. Add foreground system

### 1. Create technosphere columns

1. Read the file from user input

In [2]:
input_tech_df = pd.read_csv("user_input.csv", delimiter=",")

2. Find all exiobase activties from `A.txt`

In [3]:
df = pd.read_csv("A.txt", delimiter="\t")
activities = df.iloc[2:, 0] + "-" + df.iloc[2:, 1]
activities = activities.to_list()
activities

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline',
 'EU28-Gas/Diesel Oil',
 'EU28-Refinery Feedstocks',
 'EU28-Lubricants',
 'EU28-P- and other fertiliser',
 'EU28-Chemicals nec',
 'EU28-Biogasoline',
 'EU28-Biodiesels',
 'EU28-Cement, lime and plaster',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'EU28-Aluminium and aluminium products',
 'EU28-Copper products',
 'EU28-Electricity by coal',
 'EU28-Electricity by gas',
 'EU28-Electricity by nuclear',
 'EU28-Electricity by hydro',
 'EU28-Electricity by wind',
 'EU28-Electricity by petroleum and other oil derivatives',
 'EU28-Electricity by biomass and waste',
 'EU28-Electricity by solar photovoltaic',
 'EU28-Electricity by solar thermal',
 'EU28-Electricity by tide, wave, ocean',
 'EU28-Electricity by Geothermal',
 'EU28-Electricity nec',
 'EU28-Steam and hot water supply 

3. Get all exchanges from user's input

In [4]:
exchanges = input_tech_df.iloc[:, 0].to_list()
exchanges

['Use stage',
 'Production stage',
 'Enf of life stage',
 'RoW-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'RoW-Energy',
 'CO2 - waste - fossil - air',
 'CO2 - waste - biogenic - air',
 'CO2 - agriculture - peat decay - air']

4. Get the activities that need to be changed.

In [5]:
activity_change = list(set(activities) & set(exchanges))  # this use the property of set: a set cannot conclude duplicate elements.
activity_change

['RoW-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'RoW-Energy']

5. Select to get the data of each `activity_change`

In [6]:
activity_change_df = input_tech_df[input_tech_df["Process"].isin(activity_change)]
activity_change_df

,Process,Use stage,Production stage,Enf of life stage
3,RoW-Agriculture-Forestry-Fishing,0.0,0.0,0
4,EU28-Energy,4.0,0.0,0
5,EU28-Basic iron and steel and of ferro-alloys ...,0.0,2.4,0
6,RoW-Energy,3.5,0.0,0


6. Get the indices of activity_change

In [7]:
tech_row_index = [] # here tech_index means row index of technosphere
for act in activity_change:
    tech_row_index.append(activities.index(act))
print(tech_row_index)

[38, 1, 13, 39]


7. Build original matrix

In [8]:
raw_tech = df.iloc[2:, 2:].astype('float').to_numpy()
identity_matrix = np.identity(len(raw_tech))
tech_matrix = - (identity_matrix - raw_tech)
np.fill_diagonal(tech_matrix, -tech_matrix.diagonal())

8. Define a function to add columns (call to add one column at a time)

In [9]:
def add_column(input_column, new_column, indices):
    k = 0  # we use k to count activity_change equals to zero
    for index, data in zip(indices, input_column):
        if data == 0:
            k += 1
        new_column[index] = data
    print(f"{k} of the activity_change gives zero.")

    return new_column

9. Add each column iteratively

In [ ]:
# those are the data need to be added
activity_change_df.iloc[:, 1:]

,Use stage,Production stage,Enf of life stage
3,0.0,0.0,0
4,4.0,0.0,0
5,0.0,2.4,0
6,3.5,0.0,0


In [ ]:
# get the number of the column that need to be added.
number_of_column = activity_change_df.iloc[:, 1:].shape[1]
number_of_column

3

In [12]:
columns = []
for i in range(number_of_column):  
    column = np.zeros(len(tech_matrix))  # Create a all zero list
    columns.append(add_column(activity_change_df.iloc[:, i+1],  # By default, additional columns start from column number 1
                              column, 
                              tech_row_index))  # here again it's the row index of technosphere, not datapackage indices.

tech_columns = np.vstack(columns).T
print(tech_columns.shape)


2 of the activity_change gives zero.
3 of the activity_change gives zero.
4 of the activity_change gives zero.
(76, 3)


### 2. Create biosphere columns

1. Read user's input

In [13]:
input_bio_df = pd.read_csv("user_input.csv", delimiter=",")  # this is actually reading the same file as the add technosphere columns step.

2. Get all emissions for biosphere

In [14]:
# I copy this from our project code
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - biogenic - air", 
        "CO2 - waste - fossil - air",
        "CO2 - agriculture - peat decay - air", 
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

3. Find all emissions from user's input

In [15]:
bio_change_df = input_bio_df[input_bio_df["Process"].isin(GHG)].copy()
bio_change = bio_change_df.iloc[:, 0].to_list()
bio_change

['CO2 - waste - fossil - air',
 'CO2 - waste - biogenic - air',
 'CO2 - agriculture - peat decay - air']

4. Find the indices

In [16]:
bio_row_index = []
for bio in bio_change:
    bio_row_index.append(GHG.index(bio))

bio_row_index

[4, 3, 5]

5. Build original biosphere matrix

In [17]:
bio_df = pd.read_csv("S.txt", delimiter="\t")
# .iloc[2:, 1:]
bio_df = bio_df[bio_df.iloc[:, 0].isin(GHG)]
bio_matrix = bio_df.iloc[:, 1:]
bio_matrix

,EU28,EU28.1,EU28.2,EU28.3,EU28.4,EU28.5,EU28.6,EU28.7,EU28.8,EU28.9,...,RoW.28,RoW.29,RoW.30,RoW.31,RoW.32,RoW.33,RoW.34,RoW.35,RoW.36,RoW.37
25,69271.9534625,224902.521363,1587172.07116,48660.9000799,264966.103741,272078.927373,240242.756561,250157.081085,448857.504502,134383.541381,...,51147394.4634,277620.386813,48038.2827372,269140.508066,23946.3826027,190043.080718,1769950.15388,81146.3101981,50915.3559395,50286.2192493
26,17.9844072657,34.3942741243,619.8296864,11.9501794102,76.3234853957,80.0336423169,69.9122683148,77.5327909472,109.287554409,39.1435112998,...,6184.68117076,34.0905512004,12.1928975352,14.4986153271,2.25545832441,39.4673494252,296.253879822,20.1630038092,11.8073815707,13.0264152456
27,2.19718421163,3.01196685117,3.69253097526,1.71578825134,1.51688019166,1.55677757941,1.40469103455,1.59218945473,4.579294613,0.853479528452,...,1035.24850688,4.74953147194,1.13955663864,7.1977005388,0.519371913583,5.73557722091,55.6068916808,1.38856482356,0.637694131205,0.710205513987
69,2.13043226587,301.456635812,31379.9656223,59.2994501815,91.1452491013,77.3341780604,43.7932278174,96.7862416276,143.451412854,39.2975157085,...,42.6957140569,1.30332276205,9.96476766914,14.0108040123,8.90344177292,15.1739023148,16.3917637426,89.1575242408,20.9670138487,16.4812599126
70,1.82848728601,1757.66178494,16.958877416,35.0365130978,664.668188331,65.3006528236,222.152424537,288.193012364,142.980318874,51.4509388264,...,189.330730329,2.61016094932,9.73766422902,31.7382149792,11.5136729761,24.9240945113,14.3809378555,140.993574087,3.1117788729,52.7295912098
71,0.0269758079072,20.3928414953,0.00125794760375,0.0217202439629,0.146401638576,0.159952280928,0.153252801977,0.124777552703,0.512458533464,0.30650137405,...,4.39866159203,2.71302934484,0.197112281433,1.31413097803,0.430308952808,5.77375472514,17.1846604227,6.24102170529,3.33102492795,7.07417490613
72,0.119730927821,608.100959292,0.386101442319,1.79110398294,0.589950669195,0.719545791469,0.365643376265,0.56923642167,0.94876750247,0.609325538112,...,147.433311932,48.2725084197,9.16577431711,116.137251149,38.2088787988,117.924564192,378.653483842,137.03813274,171.248940063,134.773594853
73,0.216618044851,253.342924877,0.323371746143,0.465341733634,1.38487469554,1.55771863539,1.16745453768,1.06432757471,0.510206440808,0.386689005053,...,22.063482616,9.22241807107,2.15221812466,5.97030411535,2.88517848655,18.0890099388,66.3029055331,21.7848598906,11.4889887605,31.6175270741
74,0.20694486994,243.821137923,21.3550322084,0.278143819861,0.663525716614,0.738393109613,0.581068395198,0.495003831874,1.76909769466,0.58411613156,...,0.78608124025,0.372286741496,0.0751445603568,1.22442842332,0.389836082417,0.643302851851,1.74759708073,0.674579319882,0.279211073834,1.08194332315
75,0.00055343325484,5.47008727712,0.003266127279,0.00802216499854,0.000186051338172,0.000163108722548,0.0017652874165,0.00186425598589,0.43433741245,0.272543033889,...,18.1496951497,1.29286671614,0.630916523033,5.92490149199,3.24935287819,2.76543725181,8.79162613065,3.12688851417,1.13021200103,2.2665074386


6. Add biosphere columns

In [18]:
columns = []
for i in range(number_of_column):
    column = np.zeros(len(bio_matrix))  # Create a all zero list
    columns.append(add_column(bio_change_df.iloc[:, i+1],  # By default, additional columns start from column number 1
                              column, 
                              bio_row_index))  # here again it's the row index of technosphere, not datapackage indices.

bio_columns = np.vstack(columns).T

print(bio_columns.shape)

3 of the activity_change gives zero.
2 of the activity_change gives zero.
3 of the activity_change gives zero.
(20, 3)


### 3. Create rows (all zeros)

In [19]:
tech_rows = np.zeros((number_of_column, len(tech_matrix)))
tech_rows.shape

(3, 76)

### 4. Create foreground exchanges matrix

In [20]:
activity_change

['RoW-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Basic iron and steel and of ferro-alloys and first products thereof',
 'RoW-Energy']

In [21]:
bio_change

['CO2 - waste - fossil - air',
 'CO2 - waste - biogenic - air',
 'CO2 - agriculture - peat decay - air']

In [22]:
input_df = pd.read_csv("user_input.csv", delimiter=",")
foreground_exchanges = input_df[~input_df.iloc[:, 0].isin(activity_change+bio_change)].iloc[:, 1:].copy()
foreground_exchanges

,Use stage,Production stage,Enf of life stage
0,10.0,0.0,0
1,2.0,1.0,0
2,3.0,0.0,234


### 4. Concatenate all foreground together

1. concatenate tech_columns to technosphere matrix

In [23]:
tech_column_added = np.concatenate((tech_columns, tech_matrix), axis=1)
tech_column_added.shape

(76, 79)

2. concatenate bio_columns to biosphere matrix

In [24]:
bio_matrix_new = np.concatenate((bio_columns, bio_matrix), axis=1)
bio_matrix_new.shape

(20, 79)

3. concatenate tech_rows with forground_exchanges

In [25]:
tech_rows_foreground_exchanges = np.concatenate((tech_rows, foreground_exchanges), axis=1)
tech_rows_foreground_exchanges.shape

(3, 79)

4. concatenate tech matrix together 

In [26]:
tech_matrix_new = np.concatenate((tech_rows_foreground_exchanges, tech_column_added), axis=0)
tech_matrix_new.shape

(79, 79)